In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

/home/kumarsau/venvs/jenv/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
# Sample data
symbols = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']
strategies = ['Strategy_A', 'Strategy_B', 'Strategy_C', 'Strategy_D', 'Strategy_E']

# Generate sample data
n = 1000
data = {
    'ric': np.random.choice(symbols, n),
    'date': [datetime.now() - timedelta(days=random.randint(0, 13)) for _ in range(n)],
    'strategy': np.random.choice(strategies, n),
    'qty': np.random.randint(1, 1000, n)
}

# Create DataFrame
df = pd.DataFrame(data)

In [3]:
# Drop the time part from the date column
df['date'] = df['date'].dt.date

# print(df.head())

df = df.groupby(['ric', 'date', 'strategy'])['qty'].sum().reset_index()
df.reset_index(inplace=True)

dates = df['date'].drop_duplicates().tolist()

df_list = []
for date in dates:
    df2  = df[df['date']==date]
    df3 = df2[['ric', 'strategy', 'qty', 'date']]
    df3.columns = pd.MultiIndex.from_product([[date], ['ric', 'strategy', 'qty', 'date']])
    df3 = df3.drop('date', axis = 1, level = 1)
    # print(df3)
    df_list.append(df3)
    
df_res = df_list[0]
for i in range(len(df_list)-1):
    df_res = pd.concat([df_res, df_list[i+1]], axis=0, ignore_index=True)
    
df_sorted = df_res.copy()
for date in dates:
	df_sorted[date] = df_sorted[date].sort_values(by='ric').reset_index(drop=True)
df_sorted = df_sorted.dropna(axis = 0, how = 'all')
df_sorted = df_sorted.fillna("") 

In [4]:
from IPython.display import display, HTML

In [25]:
# Convert DataFrame to HTML
html = df_sorted.to_html()

# Create HTML with CSS for scrollable table
scrollable_html = f"""
<p style="color:rgba(0,0,0,1);">Hi. some text for you to insert</p>
<style>
    .scrollable-table {{
        max-height: 400px;
        overflow-y: scroll;
        display: block;
    }}
    table {{
        width: 100%;
        border-collapse: collapse;
    }}
    th, td {{
        border: 1px solid black;
        padding: 5px;
        text-align: left;
    }}
</style>
<div class="scrollable-table">
    {html}
</div>
"""

In [26]:
import os
import ssl
import smtplib
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [27]:
sender = "ohboy3947@gmail.com"
password = "REPLACE_PASSWORD"
receiver = "saurabhiitd3@gmail.com"

In [28]:
subject = "test dataframe4"

In [29]:
msg = MIMEMultipart('alternative')
msg['Subject'] = subject
msg['From'] = sender
msg['To'] = receiver

In [30]:
# Create the body of the message (a plain-text and an HTML version).
# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.
msg.attach(MIMEText(scrollable_html, 'html'))

In [31]:
context = ssl.create_default_context()
smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context)

In [32]:
smtp.login(sender, password)
smtp.sendmail(sender, receiver, msg.as_string())

{}